# Basic Model Interface (BMI) for streamflow routing using t-route

This t-route network was developed for use in the Next Generation National Water Model (Nextgen). Nextgen runs models with Basic Model Interface (BMI).

In [1]:
import sys
import numpy as np

sys.path.append("src/")
import bmi_troute # This is the BMI t-route that we will be running from the file: bmi_troute.py

Initialize MC only model with configuration file

In [2]:
model = bmi_troute.bmi_troute()
model.initialize(bmi_cfg_file='test/BMI/test_MC_bmi.yaml')

Set static values

In [3]:
model.set_value('segment_id', np.array([10236, 10237, 10245]))
model.set_value('segment_toid', np.array([10237, 10238, 10236]))
model.set_value('dx', np.array([5954.08922648, 5772.19680206, 1807.48910221]))
model.set_value('n', np.array([0.06, 0.06, 0.06]))
model.set_value('ncc', np.array([0.12, 0.12, 0.12]))
model.set_value('s0', np.array([0.00730121, 0.00635386, 0.00443]))
model.set_value('bw', np.array([2.63175869, 3.49006681, 2.32079768]))
model.set_value('tw', np.array([4.38626441, 5.81677835, 3.86799598]))
model.set_value('twcc', np.array([13.15879356, 17.45033484, 11.60398769]))
model.set_value('alt', np.array([1.0, 1.0, 1.0]))
model.set_value('musk', np.array([3600.0, 3600.0, 3600.0]))
model.set_value('musx', np.array([0.2, 0.2, 0.2]))
model.set_value('cs', np.array([0.62431187, 0.56729497, 0.64251596]))

In [4]:
model.get_time_step()

300

In [5]:
model.get_value('channel_exit_water_x-section__volume_flow_rate')

array([0., 0., 0.])

Set values

In [6]:
model.set_value('land_surface_water_source__volume_flow_rate', np.array([1000.0,1000.0,1000.0]))

In [ ]:
model.update()

In [10]:
model.get_value('channel_exit_water_x-section__volume_flow_rate')

10236     7.292922
10237     7.033934
10245    18.583834
Name: (0, q), dtype: float32

In [11]:
model.get_current_time()

300.0

In [12]:
model.initialize(bmi_cfg_file='test/BMI/test_MC_bmi.yaml')

Set static values

In [13]:
model.set_value('segment_id', np.array([10236, 10237, 10245]))
model.set_value('segment_toid', np.array([10237, 10238, 10236]))
model.set_value('dx', np.array([5954.08922648, 5772.19680206, 1807.48910221]))
model.set_value('n', np.array([0.06, 0.06, 0.06]))
model.set_value('ncc', np.array([0.12, 0.12, 0.12]))
model.set_value('s0', np.array([0.00730121, 0.00635386, 0.00443]))
model.set_value('bw', np.array([2.63175869, 3.49006681, 2.32079768]))
model.set_value('tw', np.array([4.38626441, 5.81677835, 3.86799598]))
model.set_value('twcc', np.array([13.15879356, 17.45033484, 11.60398769]))
model.set_value('alt', np.array([1.0, 1.0, 1.0]))
model.set_value('musk', np.array([3600.0, 3600.0, 3600.0]))
model.set_value('musx', np.array([0.2, 0.2, 0.2]))
model.set_value('cs', np.array([0.62431187, 0.56729497, 0.64251596]))

Set dynamic values

In [14]:

model.set_value('land_surface_water_source__volume_flow_rate', np.array([1000.0,1000.0,1000.0]))

In [ ]:
model.update_until(3600) # input value is seconds. 1800s = 0.5 hours

In [16]:
model.get_current_time()

3600.0

In [17]:

model.get_value('channel_exit_water_x-section__volume_flow_rate')

10236     974.055908
10237    1476.090332
10245     987.471130
Name: (11, q), dtype: float32

In [19]:
model.finalize()